In [9]:
# Project PyAutomata Performance Testing
"""
This notebook exists as a means to evaluate Rust v. Python logic computation.
"""
# Python Modules
from time import perf_counter

# Third-Party Modules
import matplotlib.pyplot as plt

# Local Modules
from pyautomata import Automata, Canvas
from pyautomata.handlers import RUST_AVAILABLE

# Global Variables
automata = Automata(30)
canvas = Canvas(automata, generate=False)

# Functions
def generate_test(canvas: Canvas, columns: int, python:bool):
    start_time = perf_counter()
    canvas.columns = columns
    canvas.generate(force_python=python)
    time_taken = perf_counter() - start_time
    return time_taken

In [ ]:
# Generation Calculation
"""
Rust is near instant on modern computers for values less than 10000.
Python is significantly slower and 10000 will start to become several minutes,
  with higher values quickly taking hours.

A print statement will track the progress as it works through the dataset.
"""
column_list = [100, 250, 1000, 2500, 5000, 10000]

python_generate: dict[int, float] = {}
rust_generate: dict[int, float] = {}

# Rust Tests
if RUST_AVAILABLE:
    for columns in column_list:
        print(f'Working on: {columns} in Rust --------', end='\r')
        rust_generate[columns] = generate_test(canvas, columns, False)

# Python Tests
for columns in column_list:
    print(f'Working on: {columns} in Python --------', end='\r')
    python_generate[columns] = generate_test(canvas, columns, True)

print('Completed all generation ---------')

In [ ]:
# Plot Calculation time

plt.plot(rust_generate.keys(), rust_generate.values(), marker='o', linestyle='-')
plt.plot(python_generate.keys(), python_generate.values(), marker='o', linestyle='-')
plt.xlabel('Columns')
plt.ylabel('Time taken (s)')
plt.title('Calculation Time')
plt.grid(True)
plt.show()

In [ ]:
# Rust Speed Ratio

rust_boost = {}

if not RUST_AVAILABLE:
    raise ValueError('Rust must be available for this test')

for columns in column_list:
    rust_boost[columns] = python_generate[columns] / rust_generate[columns]


plt.plot(rust_boost.keys(), rust_boost.values(), marker='o', linestyle='-')
plt.xlabel('Columns')
plt.ylabel('Speed Boost (x)')
plt.title('Rust Speed Boost Ratio')
plt.grid(True)
plt.show()